In [ ]:
import gcsfs
import duckdb
import json
import pandas as pd  # <-- novo

BUCKET = "teste-waha"
PREFIX = "raw/waha_events/"

fs = gcsfs.GCSFileSystem()

# lista e filtra NDJSON
arquivos = [a for a in fs.find(f"{BUCKET}/{PREFIX}") if a.endswith(".ndjson")]
print(f"Total de arquivos: {len(arquivos)}")

# lê tudo em memória
rows = []
for path in arquivos:
    with fs.open(path, "r") as f:
        for line in f:
            try:
                rows.append(json.loads(line))
            except json.JSONDecodeError:
                pass

print(f"Total de registros lidos: {len(rows)}")

# converte para DataFrame (pode ter colunas aninhadas como dict/objeto)
df = pd.json_normalize(rows)  # usa json_normalize para “achatar” onde der




In [ ]:
# consulta com DuckDB
con = duckdb.connect()
con.register("mensagens", df)
res = con.execute("""SELECT
message_id as id,
"payload.timestamp" as timestamp,
"payload.from" as sender,
"payload.body" as body
FROM mensagens LIMIT 5""").df()
print(res)